In [1]:
import re
import json
import math
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [2]:
locations = pd.DataFrame([
    # Popeyes Manchester (examples from official Popeyes UK restaurant pages)
    {"brand":"Popeyes", "country":"UK", "region":"North West", "city":"Manchester", "site_name":"Manchester Piccadilly", "source":"popeyesuk.com"},
    {"brand":"Popeyes", "country":"UK", "region":"North West", "city":"Manchester", "site_name":"Manchester Bury New Road", "source":"popeyesuk.com"},
    
    # Chick-fil-A UK (no Manchester site listed; include nearest/UK sites for "rest of UK" comparison)
    {"brand":"Chick-fil-A", "country":"UK", "region":"Yorkshire", "city":"Leeds", "site_name":"Chick-fil-A Leeds", "source":"chick-fil-a.co.uk"},
    {"brand":"Chick-fil-A", "country":"UK", "region":"Northern Ireland", "city":"Templepatrick", "site_name":"Chick-fil-A Templepatrick (Applegreen)", "source":"chick-fil-a.co.uk"},
    {"brand":"Chick-fil-A", "country":"UK", "region":"Northern Ireland", "city":"Lisburn", "site_name":"Chick-fil-A Lisburn South (Applegreen)", "source":"chick-fil-a.co.uk"},
])
locations


,brand,country,region,city,site_name,source
0,Popeyes,UK,North West,Manchester,Manchester Piccadilly,popeyesuk.com
1,Popeyes,UK,North West,Manchester,Manchester Bury New Road,popeyesuk.com
2,Chick-fil-A,UK,Yorkshire,Leeds,Chick-fil-A Leeds,chick-fil-a.co.uk
3,Chick-fil-A,UK,Northern Ireland,Templepatrick,Chick-fil-A Templepatrick (Applegreen),chick-fil-a.co.uk
4,Chick-fil-A,UK,Northern Ireland,Lisburn,Chick-fil-A Lisburn South (Applegreen),chick-fil-a.co.uk


In [3]:
menu_cols = [
    "brand","site_name","category","item_name",
    "is_combo","includes","price_gbp","channel","source_url"
]
menu = pd.DataFrame(columns=menu_cols)
menu.head()


,brand,site_name,category,item_name,is_combo,includes,price_gbp,channel,source_url


In [4]:
def scrape_popeyes_official_menu(url="https://popeyesuk.com/menu"):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    text = r.text
    
    # Popeyes site sometimes returns text/plain (good for simple parsing)
    # We'll grab likely item lines (very lightweight).
    lines = [ln.strip(" ·\n\r\t") for ln in text.split("\n")]
    lines = [ln for ln in lines if ln and len(ln) < 80]
    
    # heuristic: keep lines that look like items
    items = []
    for ln in lines:
        if any(k in ln.lower() for k in ["sandwich","wrap","wings","tenders","classic","superstack","honey"]):
            items.append(ln)
    items = sorted(set(items))
    return items

popeyes_items = scrape_popeyes_official_menu()
popeyes_items[:20], len(popeyes_items)


([], 0)

In [5]:
for site in ["Manchester Piccadilly", "Manchester Bury New Road"]:
    for item in popeyes_items:
        menu.loc[len(menu)] = {
            "brand":"Popeyes",
            "site_name":site,
            "category":"Unknown (from official menu)",
            "item_name":item,
            "is_combo":False,
            "includes":None,
            "price_gbp":np.nan,
            "channel":"in_store",
            "source_url":"https://popeyesuk.com/menu"
        }

menu.tail()


,brand,site_name,category,item_name,is_combo,includes,price_gbp,channel,source_url


In [6]:
def scrape_cfa_uk_menu(url="https://www.chick-fil-a.co.uk/menu/"):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "lxml")
    
    # This is intentionally generic: collect visible text links that look like menu items.
    texts = []
    for a in soup.select("a"):
        t = " ".join(a.get_text(" ", strip=True).split())
        if not t:
            continue
        if len(t) > 60:
            continue
        # common menu-ish keywords
        if any(k in t.lower() for k in ["sandwich","nuggets","fries","salad","milkshake","lemonade","sauce","wrap"]):
            texts.append(t)
    return sorted(set(texts))

cfa_items = scrape_cfa_uk_menu()
cfa_items[:25], len(cfa_items)


(['Chick-fil-A ® Chicken Sandwich',
  'Grilled Nuggets',
  'Salads',
  'Sauces & dressings',
  'Spicy Deluxe Sandwich'],
 5)

In [7]:
for site in ["Chick-fil-A Leeds", "Chick-fil-A Templepatrick (Applegreen)", "Chick-fil-A Lisburn South (Applegreen)"]:
    for item in cfa_items:
        menu.loc[len(menu)] = {
            "brand":"Chick-fil-A",
            "site_name":site,
            "category":"Unknown (from official menu)",
            "item_name":item,
            "is_combo":False,
            "includes":None,
            "price_gbp":np.nan,
            "channel":"in_store",
            "source_url":"https://www.chick-fil-a.co.uk/menu/"
        }


In [8]:
deals = pd.DataFrame([
    # Example rows (edit these based on what you see today on Just Eat / Uber Eats / Deliveroo)
    {"brand":"Popeyes","site_name":"Manchester Piccadilly","deal_name":"Chicken Sandwich Meal","includes":"1 sandwich + 1 regular side + 1 drink + 1 dip","price_gbp":11.49,"channel":"delivery_platform","source_url":"(paste link)"},
    {"brand":"Popeyes","site_name":"Manchester Piccadilly","deal_name":"Chicken Sandwich Meal for 2","includes":"2 sandwiches + sides + drinks (varies)","price_gbp":25.99,"channel":"delivery_platform","source_url":"(paste link)"},
])
deals


,brand,site_name,deal_name,includes,price_gbp,channel,source_url
0,Popeyes,Manchester Piccadilly,Chicken Sandwich Meal,1 sandwich + 1 regular side + 1 drink + 1 dip,11.49,delivery_platform,(paste link)
1,Popeyes,Manchester Piccadilly,Chicken Sandwich Meal for 2,2 sandwiches + sides + drinks (varies),25.99,delivery_platform,(paste link)


In [9]:
for _, row in deals.iterrows():
    menu.loc[len(menu)] = {
        "brand":row["brand"],
        "site_name":row["site_name"],
        "category":"Meal Deal / Shop Pack",
        "item_name":row["deal_name"],
        "is_combo":True,
        "includes":row["includes"],
        "price_gbp":row["price_gbp"],
        "channel":row["channel"],
        "source_url":row["source_url"]
    }

menu[menu["is_combo"] == True]


,brand,site_name,category,item_name,is_combo,includes,price_gbp,channel,source_url
15,Popeyes,Manchester Piccadilly,Meal Deal / Shop Pack,Chicken Sandwich Meal,True,1 sandwich + 1 regular side + 1 drink + 1 dip,11.49,delivery_platform,(paste link)
16,Popeyes,Manchester Piccadilly,Meal Deal / Shop Pack,Chicken Sandwich Meal for 2,True,2 sandwiches + sides + drinks (varies),25.99,delivery_platform,(paste link)


In [10]:
def segment_tags(item_name, includes):
    s = (item_name or "").lower() + " " + (includes or "").lower()
    tags = set()
    if any(k in s for k in ["for 2","share","sharer","family","bucket"]):
        tags.add("groups/families")
    if any(k in s for k in ["meal","deal","value","box"]):
        tags.add("value-seekers")
    if any(k in s for k in ["salad","grilled"]):
        tags.add("health-leaning")
    if any(k in s for k in ["spicy","hot","honey"]):
        tags.add("flavour-chasers")
    if any(k in s for k in ["kids"]):
        tags.add("kids")
    return sorted(tags)

menu["segment_tags"] = menu.apply(lambda r: segment_tags(r["item_name"], r["includes"]), axis=1)
menu.sample(10)


,brand,site_name,category,item_name,is_combo,includes,price_gbp,channel,source_url,segment_tags
9,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Unknown (from official menu),Spicy Deluxe Sandwich,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[flavour-chasers]
7,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Unknown (from official menu),Salads,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[health-leaning]
5,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Unknown (from official menu),Chick-fil-A ® Chicken Sandwich,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[]
13,Chick-fil-A,Chick-fil-A Lisburn South (Applegreen),Unknown (from official menu),Sauces & dressings,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[]
15,Popeyes,Manchester Piccadilly,Meal Deal / Shop Pack,Chicken Sandwich Meal,True,1 sandwich + 1 regular side + 1 drink + 1 dip,11.49,delivery_platform,(paste link),[value-seekers]
6,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Unknown (from official menu),Grilled Nuggets,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[health-leaning]
11,Chick-fil-A,Chick-fil-A Lisburn South (Applegreen),Unknown (from official menu),Grilled Nuggets,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[health-leaning]
1,Chick-fil-A,Chick-fil-A Leeds,Unknown (from official menu),Grilled Nuggets,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[health-leaning]
10,Chick-fil-A,Chick-fil-A Lisburn South (Applegreen),Unknown (from official menu),Chick-fil-A ® Chicken Sandwich,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[]
8,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Unknown (from official menu),Sauces & dressings,False,None,NaN,in_store,https://www.chick-fil-a.co.uk/menu/,[]


In [12]:
popeyes_gross_margin = 77.4 / 118.9
popeyes_gross_margin


0.6509671993271657

In [13]:
assumptions = pd.DataFrame([
    {"brand":"Popeyes","gross_margin":popeyes_gross_margin, "avg_basket_gbp":12.50, "orders_per_day":650, "fixed_costs_week_gbp":18000, "variable_cost_rate":0.06},
    # Chick-fil-A UK: no published UK store-level margin in our dataset; start with a conservative placeholder
    {"brand":"Chick-fil-A","gross_margin":0.62, "avg_basket_gbp":11.50, "orders_per_day":550, "fixed_costs_week_gbp":17500, "variable_cost_rate":0.06},
])
assumptions


,brand,gross_margin,avg_basket_gbp,orders_per_day,fixed_costs_week_gbp,variable_cost_rate
0,Popeyes,0.650967,12.5,650,18000,0.06
1,Chick-fil-A,0.620000,11.5,550,17500,0.06


In [14]:
site_model = locations.merge(assumptions, on="brand", how="left")

def profit_calc(row):
    revenue_week = row["orders_per_day"] * 7 * row["avg_basket_gbp"]
    gross_profit_week = revenue_week * row["gross_margin"]
    variable_cost_week = revenue_week * row["variable_cost_rate"]
    operating_profit_week = gross_profit_week - row["fixed_costs_week_gbp"] - variable_cost_week
    return pd.Series({
        "revenue_week_gbp": revenue_week,
        "gross_profit_week_gbp": gross_profit_week,
        "operating_profit_week_gbp": operating_profit_week,
        "revenue_year_gbp": revenue_week * 52,
        "operating_profit_year_gbp": operating_profit_week * 52,
    })

profit_table = pd.concat([site_model, site_model.apply(profit_calc, axis=1)], axis=1)
profit_table[[
    "brand","site_name","city","region",
    "revenue_week_gbp","operating_profit_week_gbp",
    "revenue_year_gbp","operating_profit_year_gbp"
]].sort_values(["brand","city"])


,brand,site_name,city,region,revenue_week_gbp,operating_profit_week_gbp,revenue_year_gbp,operating_profit_year_gbp
2,Chick-fil-A,Chick-fil-A Leeds,Leeds,Yorkshire,44275.0,7294.000000,2302300.0,379288.00000
4,Chick-fil-A,Chick-fil-A Lisburn South (Applegreen),Lisburn,Northern Ireland,44275.0,7294.000000,2302300.0,379288.00000
3,Chick-fil-A,Chick-fil-A Templepatrick (Applegreen),Templepatrick,Northern Ireland,44275.0,7294.000000,2302300.0,379288.00000
0,Popeyes,Manchester Piccadilly,Manchester,North West,56875.0,15611.259462,2957500.0,811785.49201
1,Popeyes,Manchester Bury New Road,Manchester,North West,56875.0,15611.259462,2957500.0,811785.49201


In [15]:
profit_table["market_bucket"] = np.where(profit_table["city"].eq("Manchester"), "Manchester", "Rest of UK")
summary = profit_table.groupby(["brand","market_bucket"]).agg(
    sites=("site_name","count"),
    revenue_week=("revenue_week_gbp","sum"),
    op_profit_week=("operating_profit_week_gbp","sum")
).reset_index()

summary


,brand,market_bucket,sites,revenue_week,op_profit_week
0,Chick-fil-A,Rest of UK,3,132825.0,21882.000000
1,Popeyes,Manchester,2,113750.0,31222.518923


In [16]:
menu.to_csv("menu_items_normalized.csv", index=False)
locations.to_csv("locations.csv", index=False)
deals.to_csv("deals_manual.csv", index=False)
profit_table.to_csv("profit_scenarios_by_site.csv", index=False)

print("Wrote: menu_items_normalized.csv, locations.csv, deals_manual.csv, profit_scenarios_by_site.csv")


Wrote: menu_items_normalized.csv, locations.csv, deals_manual.csv, profit_scenarios_by_site.csv
